## Necesary conditions for Knapsack problem
### Install pyQUBO from Recruit Communications Co. Ltd.
    pip install pyqubo
### Install openJij from Jij Inc.  (startup from Tohoku University)
    pip install openjij

# Solve Knapsack problem

### import pyQUBO, openJij, numpy and matplotlib

In [1]:
from pyqubo  import Array,Constraint, Placeholder
import openjij as jij
import numpy as np
import matplotlib.pyplot as plt

Array, Constrains and Placeholders are convenient classes from pyQUBO

### Prepare binary variables

In [2]:
N = 6
Wmax = 3
vartype = "BINARY"
q = Array.create("q",shape=N,vartype=vartype)
y = Array.create("y",shape=Wmax,vartype=vartype)

"q" is name of variables  
shape specifies the shape of variables as vector, matrix, or...  
vartype selects -1 or 1 by "SPIN" and 0 or 1by "BINARY"

In [3]:
print(q)

Array([Binary(q[0]), Binary(q[1]), Binary(q[2]), Binary(q[3]), Binary(q[4]), Binary(q[5])])


In [4]:
print(y)

Array([Binary(y[0]), Binary(y[1]), Binary(y[2])])


### Prepare values and weights

In [22]:
c = np.random.randint(1,N,N)
w = np.random.randint(1,N,N)
k = np.linspace(1,Wmax,Wmax)

### Define cost function

In [23]:
E1 = Constraint((np.dot(w,q)-np.dot(k,y))**2,"weight")

In [24]:
E2 = Constraint((np.sum(y)-1)**2,"y")

In [25]:
E3 = - np.dot(c,q)

In [26]:
Lam1 = Placeholder('Lam1')
Lam2 = Placeholder('Lam2')
E_cost = Lam1*E1+Lam2*E2+E3

### Compile the cost function

In [27]:
model = E_cost.compile()

### Get qubo matrix

In [28]:
feed_dict = {'Lam1': 5.0,'Lam2': 10.0}
Q, offset = model.to_qubo(feed_dict=feed_dict)

### Prepare simulation of quantum annealing

In [29]:
#simulated quantum annealing
sampler = jij.SQASampler(beta=10.0, gamma=1.0, trotter=4, num_sweeps=100)
#simulated annealing
#sampler = jij.SASampler(num_sweeps=1000)

This is done by quantum Monte-Carlo simulation  
gamma = strength of quantum fluctuation  
trotter = Trotter number  
num_sweeps = length of MCS

### Let's simulate quantum annealing

In [30]:
response = sampler.sample_qubo(Q)

### Show results

In [31]:
print(response)

  q[0] q[1] q[2] q[3] q[4] q[5] y[0] y[1] y[2] energy num_oc.
0    0    0    0    0    1    0    0    0    1    5.0       1
['BINARY', 1 rows, 1 samples, 9 variables]


### minimum sample

In [32]:
response.record["sample"]

array([[0, 0, 0, 0, 1, 0, 0, 0, 1]], dtype=int8)

### decode solution through pyQUBO

In [37]:
dsol, broken, energy = model.decode_solution(response.record["sample"][0], feed_dict = feed_dict, vartype=vartype)

In [38]:
print(dsol["q"])

{0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0}


In [39]:
print(dsol["y"])

{0: 0, 1: 0, 2: 1}
